In [1]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Initialize PEGASUS model and tokenizer
model_name = 'google/pegasus-xsum'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces/newlines with a single space
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    return text

def summarize_text(text, max_length=1000, min_length=40):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def split_into_sections(text, section_length=1000, overlap=200):
    text = preprocess_text(text)
    sections = []
    start = 0
    while start < len(text):
        end = start + section_length
        sections.append(text[start:end])
        start = end - overlap
    return sections

def summarize_sections(sections):
    return [summarize_text(section) for section in sections]

def combine_summaries(summaries):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(summaries)
    similarity_matrix = cosine_similarity(tfidf_matrix)
    sorted_indices = np.argsort(similarity_matrix.sum(axis=1))
    sorted_summaries = [summaries[i] for i in sorted_indices]
    return "\n\n".join(sorted_summaries)

def main(text):
    sections = split_into_sections(text)
    summaries = summarize_sections(sections)
    final_summary = combine_summaries(summaries)
    return final_summary

In [6]:
text = """
Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The novels chronicle the lives of a young wizard, Harry Potter, and his friends, Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns Harry's conflict with Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the Ministry of Magic, and subjugate all wizards and Muggles (non-magical people).

The series was originally published in English by Bloomsbury in the United Kingdom and Scholastic Press in the United States. A series of many genres, including fantasy, drama, coming-of-age fiction, and the British school story (which includes elements of mystery, thriller, adventure, horror, and romance), the world of Harry Potter explores numerous themes and includes many cultural meanings and references.[1] Major themes in the series include prejudice, corruption, madness, and death.[2][3]

Since the release of the first novel, Harry Potter and the Philosopher's Stone, on 26 June 1997, the books have found immense popularity, positive reviews, and commercial success worldwide. They have attracted a wide adult audience as well as younger readers and are widely considered cornerstones of modern literature.[4][5] As of February 2023, the books have sold more than 600 million copies worldwide, making them the best-selling book series in history, and have been available in 85 languages.[6] The last four books consecutively set records as the fastest-selling books in history, with the final instalment selling roughly 2.7 million copies in the United Kingdom and 8.3 million copies in the United States within twenty-four hours of its release.

Warner Bros. Pictures adapted the original seven books into an eight-part namesake film series. In 2016, the total value of the Harry Potter franchise was estimated at $25 billion,[7] making it one of the highest-grossing media franchises of all time. Harry Potter and the Cursed Child is a play based on a story co-written by Rowling.

The success of the books and films has allowed the Harry Potter franchise to expand with numerous derivative works, a travelling exhibition that premiered in Chicago in 2009, a studio tour in London that opened in 2012, a digital platform on which J. K. Rowling updates the series with new information and insight, and a trilogy of spin-off films premiering in November 2016 with Fantastic Beasts and Where to Find Them, among many other developments. Themed attractions, collectively known as The Wizarding World of Harry Potter, have been built at several Universal Destinations & Experiences amusement parks around the world.
"""

formatted_summary = main(text)
print(formatted_summary)

A look at some of the world's most popular theme park attractions and how they compare to the original Harry Potter theme park in JK Rowling's Wizarding World of Harry Potter., here's a look at some of the world's most popular theme park attractions and how they compare to the original Harry Potter theme park in JK Rowling's Wizarding World of Harry Potter., here's a look at some of the world's most popular theme park attractions and how they compare to the original Harry Potter theme park in JK Rowling's Wizarding World of Harry Potter., here's a look at some of the world's most popular theme park attractions and how they compare to the original Harry Potter theme park in JK Rowling's Wizarding World of Harry Potter., here's a look at some of the world's most popular theme park attractions and how they compare to the original Harry Potter theme park in JK Rowling's Wizarding World of Harry Potter., here's a look at some of the world's most popular theme park attractions and how they c